In [20]:
import mysql.connector
import pandas as pd

In [28]:
# подключение к БД
config = {
    'host': '84.201.138.51',
    'port': 3306,
    'user': 'sl_dev_vacancy',
    'password': 'q5Hcsp@H',
    'database': 'tv_dashboards'
}

connection = mysql.connector.connect(**config)

cursor = connection.cursor()

In [29]:
# запрос для получения списка таблиц
query_get_tables = "SHOW TABLES"
cursor.execute(query_get_tables)

tables = cursor.fetchall()
for table in tables:
    print(table[0])

test_base_tv


Групируем по - New_brand, Year, Month, Week, TVCompany, Programme_genre 
Сумируем - Quantity, Consolidated_cost, TVR_All_18+


In [83]:
query_get_data_from_table = '''SELECT sum(Quantity) as sum_quantity, sum(Consolidated_cost) as sum_consolidated_cost,
                                sum(`TVR_All_18+`) as `SUM_TVR_All_18+`, count(*) as count_in_group,
                                Programme_genre, New_brand, Year, Month, Week, TVCompany
                                FROM test_base_tv 
                                GROUP BY New_brand, Year, Month, Week, TVCompany, Programme_genre
                            '''
cursor.execute(query_get_data_from_table)
data = cursor.fetchall()

In [84]:
columns = [column[0] for column in cursor.description]

In [167]:
df = pd.DataFrame(data, columns=columns)

In [168]:
# 1. Добавить в основной df столбец, где на стоимость начисляется НДС(VAT).
df['VAT'] = df.apply(lambda row: float(row['sum_consolidated_cost']) * 1.18 if row['Year'] <= 2018 else float(row['sum_consolidated_cost']) * 1.2  , axis=1)

In [169]:
# 2. По полю TVCompany убрать из названия каналов словосочетание “(СЕТЕВОЕ ВЕЩАНИЕ)”.
df['TVCompany'] = df['TVCompany'].apply(lambda company: str(company).rsplit('(СЕТЕВОЕ ВЕЩАНИЕ)')[0].rstrip())

In [170]:
# 3. Создать новый df, где по брендам/годам будет рассчитан средний недельный TVR_All_18+.
df_mean_sum_tvr_all_18 = df.groupby(['New_brand','Year', 'Week']).agg({'SUM_TVR_All_18+': 'mean'})
df_mean_sum_tvr_all_18.rename(columns = {'SUM_TVR_All_18+':' SUM_TVR_All_18+(mean)'}, inplace = True)

In [182]:
# 4. Cоздать новый df, где по годам/телекомпаниям будет посчитан суммарный Consolidated_cost.
df_sum_consolidated_cost = df.groupby(['Year', 'TVCompany']).agg({'sum_consolidated_cost': 'sum'})
df_sum_consolidated_cost.rename(columns = {'sum_consolidated_cost':' sum_consolidated_cost(sum)'}, inplace = True)

In [187]:
# 5. Cоздать новый df, где по жанрам программ будет рассчитана средняя стоимость выхода рекламы с НДС. Жанры ранжированы по убыванию стоимости.
df_mean_const_vat = df.groupby('Programme_genre').agg({'VAT' : 'mean'}).sort_values(by='VAT', ascending=False)
df_mean_const_vat.rename(columns = {'VAT':' VAT(mean)'}, inplace = True)

Результаты 

In [176]:
# Основной df
df

,sum_quantity,sum_consolidated_cost,SUM_TVR_All_18+,count_in_group,Programme_genre,New_brand,Year,Month,Week,TVCompany,VAT
0,3,15521.00,0.3210,3,Музыкальная программа,АФОБАЗОЛ,2017,1,1,МУЗ ТВ,18314.78
1,22,78804.00,0.7337,22,Прочее,АФОБАЗОЛ,2017,1,2,ANIMAL PLANET,92988.72
2,24,0.00,0.1548,24,Прочее,АФОБАЗОЛ,2017,1,2,БОБЁР,0.00
3,16,725905.00,10.0166,16,Кинопрограмма,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,856567.90
4,2,45076.00,0.5084,2,Развлекательная программа,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,53189.68
...,...,...,...,...,...,...,...,...,...,...,...
60713,5,481991.00,2.1279,5,Кинофильм,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,578389.20
60714,4,427357.00,2.4019,4,Познавательная программа,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,512828.40
60715,1,96719.00,0.6290,1,Социально-политическая программа,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,116062.80
60716,13,1695651.00,8.8612,13,Телесериал,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,2034781.20


In [184]:
# df задание №3
df_mean_sum_tvr_all_18 

SUM_TVR_All_18+(mean)
New_brand                    Year Week                       
АФОБАЗОЛ                     2017 1                     0.321
                                  2                  5.963034
                                  3                  3.578233
                                  4                  4.682577
                                  5                  3.270063
...                                                       ...
ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ 2023 6                    4.0768
                                  11                 5.788867
                                  12                 8.268828
                                  13                 6.609021
                                  15                    0.808

[1243 rows x 1 columns]

In [185]:
# df задание №4
df_sum_consolidated_cost

sum_consolidated_cost(sum)
Year TVCompany                                     
2017 24_DOC                                    0.00
     2X2                                 7624684.48
     ANIMAL PLANET                       4886663.00
     BRIDGE                                 2139.00
     BRIDGE РУССКИЙ ХИТ                    76226.00
...                                             ...
2023 ТВ-3 (ИНТЕРНЕТ)                           0.00
     ТНТ                               175540370.00
     ТНТ (ИНТЕРНЕТ)                            0.00
     ФЕНИКС+ КИНО                            561.00
     Ю                                  20182778.00

[434 rows x 1 columns]

In [188]:
# df задание №5
df_mean_const_vat

,VAT(mean)
Programme_genre,
Кинопрограмма,882355.538925
Развлекательная программа,476343.921354
Телесериал,451758.079069
Прочее,417847.632320
Кинофильм,269262.435507
Социально-политическая программа,257348.938250
Анимация,229361.417920
Музыкальная программа,227872.270531
Новости,170614.017075
